In [5]:
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup


def extract_text(el):
    if el:
        return el.text.strip()
    else:
        return ''
        

def get_company_from_result(result):
    return extract_text(result.find('span', {'class' : 'company'}))

def get_location_from_result(result):
    return  extract_text(result.find('span', {'class' : 'location'}))

def get_summary_from_result(result):
    return  extract_text(result.find('span', {'class' : 'summary'}))

def get_title_from_result(result):
    return result.find('a', {'data-tn-element' : 'jobTitle'}).text.strip()

def get_salary_from_result(result):
    salary_table = result.find('td', {'class' : 'snip'})
    if salary_table:
        snip = salary_table.find('nobr')
        if snip:
            return snip.text.strip()
        
    return None

url = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
rows = []

for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle']):
    for start in range(10, 8000, 10):
        r = requests.get(url.format(city, start))
        soup = BeautifulSoup(r.content)
        results = soup.findAll('div', { "class" : "result" })
        for result in results:
            if result:
                row = {}
                row['title'] = get_title_from_result(result)
                row['company'] = get_company_from_result(result)
                row['summary'] = get_summary_from_result(result)
                row['salary'] = get_salary_from_result(result)
                row['city'] = city
                rows.append(row)

import pandas as pd
data = pd.DataFrame.from_records(rows)

salary_data = data[data.salary.notnull()]
salary_data = salary_data[~(salary_data.salary.astype('str').str.contains('hour'))]

ConnectionError: ('Connection aborted.', error(54, 'Connection reset by peer'))